In [1]:
import numpy as np
import networkx as nx
import os
import pandas as pd
from functions import *

In [3]:
#select your folder
ad_folder_path = r"your folder path"
cn_folder_path = r"your folder path"
ad_csv_files = [f for f in os.listdir(ad_folder_path) if f.endswith('.csv')]
cn_csv_files = [f for f in os.listdir(cn_folder_path) if f.endswith('.csv')]

ad_matrices = get_matrices(ad_folder_path,ad_csv_files)
cn_matrices = get_matrices(cn_folder_path,cn_csv_files)

#select atlas file
df = pd.read_csv(r"your atlas file")

ad_sub_list = get_sub_list(ad_matrices,df)
group_ad_MF,group_ad_FP,group_ad_DM = get_groups(ad_sub_list)

cn_sub_list = get_sub_list(cn_matrices,df)
group_cn_MF,group_cn_FP,group_cn_DM = get_groups(cn_sub_list)

window_size = W # set your window size
stride = s # set a stride
cut_ad_MF = get_cut_data(group_ad_MF,window_size,stride)
cut_ad_FP = get_cut_data(group_ad_FP,window_size,stride)
cut_ad_DM = get_cut_data(group_ad_DM,window_size,stride)
cut_cn_MF = get_cut_data(group_cn_MF,window_size,stride)
cut_cn_FP = get_cut_data(group_cn_FP,window_size,stride)
cut_cn_DM = get_cut_data(group_cn_DM,window_size,stride)

In [4]:
#snbg method
#change data's construction
zipped_list_ad_MF = list(zip(*cut_ad_MF))
zipped_list_ad_FP = list(zip(*cut_ad_FP))
zipped_list_ad_DM = list(zip(*cut_ad_DM))
zipped_list_cn_MF = list(zip(*cut_cn_MF))
zipped_list_cn_FP = list(zip(*cut_cn_FP))
zipped_list_cn_DM = list(zip(*cut_cn_DM))
window_sub_ad_MF = [list(x) for x in zipped_list_ad_MF]
window_sub_ad_FP = [list(x) for x in zipped_list_ad_FP]
window_sub_ad_DM = [list(x) for x in zipped_list_ad_DM]
window_sub_cn_MF = [list(x) for x in zipped_list_cn_MF]
window_sub_cn_FP = [list(x) for x in zipped_list_cn_FP]
window_sub_cn_DM = [list(x) for x in zipped_list_cn_DM]

ad_thresh_MF = thresh_list_ad(window_sub_ad_MF,window_sub_cn_MF)
ad_thresh_FP = thresh_list_ad(window_sub_ad_FP,window_sub_cn_FP)
ad_thresh_DM = thresh_list_ad(window_sub_ad_DM,window_sub_cn_DM)
cn_thresh_MF = thresh_list_cn(window_sub_cn_MF)
cn_thresh_FP = thresh_list_cn(window_sub_cn_FP)
cn_thresh_DM = thresh_list_cn(window_sub_cn_DM)


ad_thresh_cut_MF = turn_construct(ad_thresh_MF)
ad_thresh_cut_FP = turn_construct(ad_thresh_FP)
ad_thresh_cut_DM = turn_construct(ad_thresh_DM)
cn_thresh_cut_MF = turn_construct(cn_thresh_MF)
cn_thresh_cut_FP = turn_construct(cn_thresh_FP)
cn_thresh_cut_DM = turn_construct(cn_thresh_DM)

Degree

In [5]:
#degree calculate
ad_degree_DM = degree_matrix_list(ad_thresh_cut_DM)
ad_degree_MF = degree_matrix_list(ad_thresh_cut_MF)
ad_degree_FP = degree_matrix_list(ad_thresh_cut_FP)
cn_degree_DM = degree_matrix_list(cn_thresh_cut_DM)
cn_degree_MF = degree_matrix_list(cn_thresh_cut_MF)
cn_degree_FP = degree_matrix_list(cn_thresh_cut_FP)

#network's degree features
list_degree_DM = []
list_degree_FP = []
list_degree_MF = []
degree_threshold_list = []

for i in range(len(ad_degree_DM)):
    feature_DM = calculate_feature(ad_degree_DM[i],cn_degree_DM[i])
    list_degree_DM.append(feature_DM)
for i in range(len(ad_degree_FP)):
    feature_FP = calculate_feature(ad_degree_FP[i],cn_degree_FP[i])
    list_degree_FP.append(feature_FP)
for i in range(len(ad_degree_MF)):
    feature_MF = calculate_feature(ad_degree_MF[i],cn_degree_MF[i])
    list_degree_MF.append(feature_MF)

for i in range(len(ad_degree_DM)):
    threshold_feature_ad = np.concatenate([ad_degree_DM[i],ad_degree_FP[i],ad_degree_MF[i]],axis=1)
    threshold_feature_cn = np.concatenate([cn_degree_DM[i],cn_degree_FP[i],cn_degree_MF[i]],axis=1)
    threshold_feature = calculate_feature(threshold_feature_ad,threshold_feature_cn)
    degree_threshold_list.append(threshold_feature)

'''DM'''
feature_degree_DM = np.vstack((list_degree_DM))
'''FP'''
feature_degree_FP = np.vstack((list_degree_FP))
'''MF'''
feature_degree_MF = np.vstack((list_degree_MF))
'''DM+FP+MF'''
feature_degree = np.vstack((degree_threshold_list))

In [ ]:
#set your cost
libsvm(feature_degree_DM,cost)
libsvm(feature_degree_FP,cost)
libsvm(feature_degree_MF,cost)
libsvm(feature_degree,cost)

Clustering

In [13]:
#network's clustering features
ad_clustering_MF = get_clustering_list(ad_thresh_cut_MF)
ad_clustering_DM = get_clustering_list(ad_thresh_cut_DM)
ad_clustering_FP = get_clustering_list(ad_thresh_cut_FP)
cn_clustering_MF = get_clustering_list(cn_thresh_cut_MF)
cn_clustering_DM = get_clustering_list(cn_thresh_cut_DM)
cn_clustering_FP = get_clustering_list(cn_thresh_cut_FP)

ad_c_DM = concatenate_clustering_feature(ad_clustering_DM)
ad_c_FP = concatenate_clustering_feature(ad_clustering_FP)
ad_c_MF = concatenate_clustering_feature(ad_clustering_MF)
ad_clustering_feature = np.concatenate([ad_c_DM,ad_c_FP,ad_c_MF],axis=1)

cn_c_DM = concatenate_clustering_feature(cn_clustering_DM)
cn_c_FP = concatenate_clustering_feature(cn_clustering_FP)
cn_c_MF = concatenate_clustering_feature(cn_clustering_MF)
cn_clustering_feature = np.concatenate([cn_c_DM,cn_c_FP,cn_c_MF],axis=1)

clustering_snbg_DM = np.vstack(calculate_feature(ad_c_DM,cn_c_DM))
clustering_snbg_FP = np.vstack(calculate_feature(ad_c_FP,cn_c_FP))
clustering_snbg_MF = np.vstack(calculate_feature(ad_c_MF,cn_c_MF))
clustering_snbg = np.vstack(calculate_feature(ad_clustering_feature,cn_clustering_feature))


In [ ]:
#set your cost
libsvm(clustering_snbg_DM,cost)
libsvm(clustering_snbg_FP,cost)
libsvm(clustering_snbg_MF,cost)
libsvm(clustering_snbg,cost)
